In [2]:
import os
import pickle
import jsonlines
import pandas as pd
import numpy as np
import json
import copy
from tqdm import tqdm

In [3]:
prompt_template = "The beauty item has following attributes: \n Its name is <TITLE>; Its brand is <BRAND>; Its price is <PRICE> dollars. \n"
feat_template = "This item belongs to the following categories: <CATEGORIES>. \n"
desc_template = "This item has the following description: <DESCRIPTION>. \n"

In [4]:
data = json.load(open("./processed/item2attributes.json", "r"))

In [5]:
all_feats = []

for user, user_attris in data.items():
    for feat_name in user_attris.keys():
        if feat_name not in all_feats:
            all_feats.append(feat_name)

In [6]:
print(all_feats)

['asin', 'related', 'title', 'price', 'salesRank', 'imUrl', 'brand', 'categories', 'description']


In [7]:
def get_attri(item_str, attri, item_info):

    if attri not in item_info.keys():
        new_str = item_str.replace(f"<{attri.upper()}>", "unknown")
    else:
        new_str = item_str.replace(f"<{attri.upper()}>", str(item_info[attri]))

    return new_str

In [8]:
def get_feat(item_str, feat, item_info):

    if feat not in item_info.keys():
        return ""
    
    assert isinstance(item_info[feat], list)
    feat_str = ""
    for meta_feat in item_info[feat][0]:
        feat_str = feat_str + meta_feat + "; "
    new_str = item_str.replace(f"<{feat.upper()}>", feat_str)

    if len(new_str) > 2048: # avoid exceed the input length limitation
        return new_str[:2048]

    return new_str

In [9]:
item_data = {}
for key, value in tqdm(data.items()):
    item_str = copy.deepcopy(prompt_template)
    item_str = get_attri(item_str, "title", value)
    item_str = get_attri(item_str, "brand", value)
    item_str = get_attri(item_str, "date", value)
    item_str = get_attri(item_str, "price", value)

    feat_str = copy.deepcopy(feat_template)
    feat_str = get_feat(feat_str, "categories", value)
    desc_str = copy.deepcopy(desc_template)
    desc_str = get_attri(desc_str, "description", value)
    
    item_data[key] = item_str + feat_str + desc_str

  0%|          | 0/57289 [00:00<?, ?it/s]

100%|██████████| 57289/57289 [00:00<00:00, 95626.68it/s]


In [22]:
item_data["1304351475"]


'The beauty item has following attributes: \n Its name is NEW EUROPEAN COLLECTION Too Faced Natural Eyes Shadow Palette 9 Colors; Its brand is Omagazee; Its price is 33.99 dollars. \nThis item belongs to the following categories: Beauty; Makeup; Eyes; Eye Shadow; . \nThis item has the following description: Too Faced *Natural Eyes Shadow Palette* \n\nColors include: Heaven, Silk Teddy, Nude Beach, Velvet Revolver, Push-up, Honey Pot, Sexspresso, Erotica, and Cocoa Puff. \n\nCollectible tin version \n\nNew in Box \n\nFull Size. \n'

In [15]:
data["1304351475"]

{'asin': '1304351475',
 'related': {'also_bought': ['B00AVUE1S6',
   'B00AYCOLMC',
   'B0013CSGBE',
   'B004FM6L0C',
   'B00C8TMB2Q',
   'B006JZ0L7C',
   'B0057N3OOY',
   'B002DP19ZU',
   'B0057N3TPS',
   'B006JZ0IRA',
   'B004YUIR9I',
   'B005628FD6',
   'B004C86GU4',
   'B003TNVSTW',
   'B004C9TJ3O',
   'B00FH2I5OE',
   'B00591JKVA',
   'B008PQ5YHK',
   'B002DP75R6',
   'B00EU6NK66',
   'B00IPQO0W0',
   'B006HMARMG',
   'B003TNY0WE',
   'B0092GWSFS',
   'B00AVUDTW0',
   'B00DJC5HK4',
   'B00GR4TPW2',
   'B00ITCAR40',
   'B00L76XBCK',
   'B008FK6VUA',
   'B0046NPGZQ',
   'B00BUO40VA',
   'B00DO40YXM',
   'B006HRVFMM',
   'B008JBIH9I',
   'B002DP1A18',
   'B00AMYNT8E',
   'B00D6DJGOE',
   'B0072CVHJC',
   'B00BUO4164',
   'B001P72H9O',
   'B004TSFE28',
   'B006GK5ZEY',
   'B006L7SF4O',
   'B0092SVTGA',
   'B007SEEAEI',
   'B00FPW9BEO',
   'B009GLQG6Q',
   'B005OZFV50',
   'B007HKOVGA',
   'B0030HMQTS',
   'B005OZJYUS',
   'B007GMJ8Z8',
   'B005OZF1S2',
   'B006JZ0IK2',
   'B006KTTGC8',

In [ ]:
json.dump(item_data, open("./processed/item_str.json", "w"))

In [10]:
item_data = json.load(open("./processed/item_str.json", "r"))

FileNotFoundError: [Errno 2] No such file or directory: './processed/item_str.json'

In [ ]:
import jsonlines

def save_data(data_path, data):
    '''write all_data list to a new jsonl'''
    with jsonlines.open("./handled/"+ data_path, "w") as w:
        for meta_data in data:
            w.write(meta_data)

id_map = json.load(open("./handled/id_map.json", "r"))["item2id"]
json_data = []
for key, value in item_data.items():
    json_data.append({"input": value, "target": "", "item": key, "item_id": id_map[key]})

save_data("item_str.jsonline", json_data)

In [ ]:
import requests
import json

url = ""

payload = json.dumps({
   "model": "text-embedding-ada-002",
   "input": "The food was delicious and the waiter..."
})
headers = {
   'Authorization': '',
   'User-Agent': 'Apifox/1.0.0 (https://apifox.com)',
   'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

In [ ]:
def get_response(prompt):
    url = ""

    payload = json.dumps({
    "model": "text-embedding-ada-002",
    "input": prompt
    })
    headers = {
    'Authorization': '',
    'User-Agent': 'Apifox/1.0.0 (https://apifox.com)',
    'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    re_json = json.loads(response.text)

    return re_json["data"][0]["embedding"]

In [ ]:
item_emb = {}

In [ ]:
value_list = []

for key, value in tqdm(item_data.items()):
    if len(value) > 4096:
        value_list.append(key)

In [ ]:
if os.path.exists("./handled/item_emb.pkl"):    # check whether some item emb exist in cache
    item_emb = pickle.load(open("./handled/item_emb.pkl", "rb"))

In [ ]:
count = 1
while 1:    # avoid broken due to internet connection
    if len(item_emb) == len(item_data):
        break
    try:
        for key, value in tqdm(item_data.items()):
            if key not in item_emb.keys():
                if len(value) > 4096:
                    value = value[:4095]
                item_emb[key] = get_response(value)
                count += 1
    except:
        pickle.dump(item_emb, open("./handled/item_emb.pkl", "wb"))
    

In [ ]:
len(item_emb)

In [ ]:
id_map = json.load(open("./handled/id_map.json", "r"))["id2item"]
emb_list = []
for id in range(1, len(item_emb)+1):
    meta_emb = item_emb[id_map[str(id)]]
    emb_list.append(meta_emb)

emb_list = np.array(emb_list)
pickle.dump(emb_list, open("./handled/itm_emb_np.pkl", "wb"))